In [1]:
import pandas as pd
import numpy as np
import time
import resource
from os import walk
from datetime  import datetime,timedelta,date
import multiprocessing as mp
import os
import re
import subprocess
import matplotlib.pyplot as plt
# from tld import get_tld
import shutil
import csv
import seaborn as sns
import pickle
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

def save_obj(obj, file_name):    
    file_path = 'pyobj/' + file_name
    file_save = open(file_path, 'wb')
    pickle.dump(obj, file_save, pickle.HIGHEST_PROTOCOL)
    file_save.close()

def load_obj(file_name):
    path = 'pyobj/' + file_name
    file_save = open(path, 'rb')
    return pickle.load(file_save)

In [2]:
def read_data_file(f):
    s = open(f, 'r').read().split('|')
    song_type = s[0]
    song_data = s[1]
    
    return (song_type, song_data)

In [5]:
types = []
data = []

f = os.walk('/home/nghia/github/Music_classification_by_lyric/NCT_Crawler/Song')
for fi in f:
    f = fi
    break
for fi in f[2]:
    try:
        content = read_data_file(f[0]+'/'+fi)
        types.append(content[0])
        data.append(content[1])
    except:
        print (f[0]+'/'+fi)

/home/nghia/github/Music_classification_by_lyric/NCT_Crawler/Song/em-la-ba-noi-cua-anh-super-version-tang-nhat-tue.txt
/home/nghia/github/Music_classification_by_lyric/NCT_Crawler/Song/tinh-doi-2-chien-thang.txt
/home/nghia/github/Music_classification_by_lyric/NCT_Crawler/Song/last-of-the-mohicans-beautiful-song-va.txt
/home/nghia/github/Music_classification_by_lyric/NCT_Crawler/Song/thong_bao_loi?utm_medium=NoSong&u_source=NCT&utm_campaign=All.txt
/home/nghia/github/Music_classification_by_lyric/NCT_Crawler/Song/vo-tuong-lai-cao-tung-anh.txt
/home/nghia/github/Music_classification_by_lyric/NCT_Crawler/Song/ta-buon-mot-doi-le-tin-huong-vu-khanh.txt
/home/nghia/github/Music_classification_by_lyric/NCT_Crawler/Song/con-kenh-ta-dao-thanh-hoa-ft-ngoc-tan.txt
/home/nghia/github/Music_classification_by_lyric/NCT_Crawler/Song/goi-do-chien-thang.txt
/home/nghia/github/Music_classification_by_lyric/NCT_Crawler/Song/nguoi-di-xay-ho-ke-go-trong-tan-ft-thanh-hoa.txt
/home/nghia/github/Music_classi

In [12]:
df1 = pd.DataFrame(pd.Series(types)).rename(columns={0:'type'})
df1['freq'] = 1
df1 = df1.groupby('type').agg({'freq':'sum'}).reset_index().sort_values('freq', ascending=False)

In [112]:
df = pd.DataFrame([types,data]).T.rename(columns={0:'type', 1:'data'})

In [113]:
df = df[~df['data'].str.contains('Hiện chưa có lời bài hát nào cho')]
df = df[df['type']!='Không Lời']

In [114]:
s = df.iloc[1:2,:].values[-1][-1]

In [110]:
# a = s.replace('    ', ' ').replace('    ', ' ').replace('   ', ' ').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ')
# a

In [134]:
def fix(s):
    for s1 in s.split('\n\n'):
        if 'Bài hát' in s1 : return s.split('\n\n')[-1].replace('\n', '.').replace('Bài Hát:', '')
    return s.replace('\n', '.')

In [135]:
# df.groupby('type').agg({'data':'count'}).reset_index().sort_values('data', ascending=False).reset_index().iloc[:,1:]

In [136]:
df['data']=df['data'].apply(fix)

In [137]:
df

,type,data
1,Trữ Tình,".. thôi,thôi ông ơi ông..."
2,Nhạc Trẻ,"Nhiều lúc rất đau em vẫn cười , cười để trách ..."
4,Nhạc Trẻ,Trái đất kia như đang bé lại.Và mỗi chúng ta n...
5,Trữ Tình,"Để tình đẹp giữa núi sông .thân yêu vội vàng, ..."
12,Trữ Tình,.. (Sáng tác: Hà Phương...
14,Pop,[Drop: Sabrina Carpenter].If you love me let m...
16,Trữ Tình,.. Bài Hát: Cơn Say Tìn...
19,Nhạc Trẻ,[ĐK:].Lời ngọt ngào trên môi.Xin người đừng nó...
20,R&B/Hip Hop/Rap,"[Outro].Baka.Yeah, we outchea.Ya don know, aye..."
21,Trữ Tình,
